# Препроцессинг диалоговых данных 

## В этом ноутбуке:

* Все вопросы-ответы (пара) заносятся в один датафрейм **raw_data** с указанием комитета (по возможности) и созыва Госдумы
* В датафрейм **processed_data** к raw_data добавлены косинусное расстояние между эмбеддингами вопроса и ответа, а также, как для вопроса, так и для ответа, доля того, что текст эмоционально-нейтральный, эмоционально-негативный, эмоционально-положительный. (а также информация о том, насколько модель уверена в своем результате, и какова доля "пустых" разговорных фраз)
* Датафрейм **processed_data_nameless** возвращает то же, что и *processed_data*, но для получения численных значений использует тексты без имен и отчеств (которые присутствуют практически в каждом диалоге в Думе).

In [1]:
!pip install torch >> None
!pip install transformers >> None
!pip install transformers sentencepiece >> None

!pip install Dostoevsky >> None
!python -m dostoevsky download fasttext-social-network-model >> None

!pip install natasha >> None

In [2]:
# библиотеки для работы с моделями платформы HuggingFace
import torch
from transformers import AutoTokenizer, AutoModel

# библиотеки для работы с эмбеддингами и текстами
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

# библиотеки для получения векторных представлений 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# библиотека Dostoevsky для задачи sentiment analysis
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

# Библиотека Natasha для получения именных представлений
# (named entity recognition)
from natasha import NamesExtractor
from natasha import MorphVocab

# библиотеки для загрузки данных
import os
import re
import glob
import requests
import ast
import pickle

# библиотеки для анализа данных
import numpy as np
import pandas as pd

# вспомогательные библиотеки
import warnings
# FutureWarnings предупреждения
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# # Удалим данные, если они уже загружены локально

# !rm -rf Scraping_stenograms

# # Загрузим данные из подготовленного репозитория Guthub

# ! git clone https://github.com/AleksandraVasilieva/Scraping_stenograms/ 

Cloning into 'Scraping_stenograms'...
remote: Enumerating objects: 683, done.
remote: Total 683 (delta 0), reused 0 (delta 0), pack-reused 683
Receiving objects: 100% (683/683), 5.72 MiB | 7.06 MiB/s, done.
Resolving deltas: 100% (302/302), done.


In [4]:
nltk.download('punkt')
nltk.download("stopwords")

# загрузка стоп слов, которые удаляются из текстов
stops = stopwords.words('russian')
# стеммер
stemmer = SnowballStemmer('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def custom_tokenizer(x):
    # приведем все символы строки в нижний регистр и токенизируем строку: 
    #разобьем ее на отдельные слова
    step1 = word_tokenize(x)
    # уберем цифры и пунктуацию
    step2 = [i.lower() for i in step1 if i.isalpha()]
    # уберем стоп-слова
    step3 = [word for word in step2 if not word in stops]
    # проведем стемминг: убрем аффиксы
    step4 = [stemmer.stem(w) for w in step3]
    return step4

In [6]:
# Токенизатор и модель для Sentiment Analysis
dostoevsky_tokenizer = RegexTokenizer()
dostoevsky_sentiment_model = FastTextSocialNetworkModel(tokenizer=dostoevsky_tokenizer)
# results = dostoevsky_sentiment_model.predict([text])

# Токенизатор и модель для получения эмбеддингов
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
rubert_embeddings_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# Модель Наташа для удаления имен
morph_vocab = MorphVocab()
natasha_names_extractor = NamesExtractor(morph_vocab)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# Функция для получения косинусного расстояния между эмбеддингами
def calculate_cosine_distance(embedding1, embedding2):
    distance = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))
    similarity = 1 - distance
    return similarity[0][0]

# Функция для получения эмбеддинга текста
def get_embeddings(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [8]:
# Функция для получения косинусного расстояния между двумя необработанными текстами
def get_cosine_similarity(question, answer):
  '''
  Функция принимает на вход вопрос и ответ,
  токенизирует, получает векторное представление
  (эмбеддинги) и возвращает косинусное расстояние.

  '''
  # Получение эмбеддингов
  embedding_question = get_embeddings(question, rubert_embeddings_model, rubert_tokenizer)
  embedding_answer = get_embeddings(answer, rubert_embeddings_model, rubert_tokenizer)

  # Получение косинусного расстояния

  cosine_distance = calculate_cosine_distance(embedding_question, embedding_answer)
  
  return cosine_distance

In [9]:
# Функция для удаления имен в строке
def nameless_text(text):
  '''
  Функция вернет текст без имен и отчеств
  '''

  # Найдем имена
  matches = list(natasha_names_extractor(text))

  # Извлечем имена из текста
  for match in matches:
      start = match.start
      end = match.stop
      text = text[:start] + text[end:]

  return text

In [18]:
# создадим датафрейм для сырого текста
raw_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Date', 'Question', 'Answer', 'Stenogram link'])

# создадим датафрейм с добавочной информацией

processed_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name',  'Date', 'Question', 'Answer', 'Stenogram link', \
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scraping_stenograms/dialogs data pickle format/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:

    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for path in files:

        commitee_name = path[len(path) - path[::-1].find('_'):][:-4]

        with open(path, 'rb') as file:
          file_content = pickle.load(file)


        # Пропустим файл, если он пустой
        if len(file_content) == 0:
          continue
          
        for chunk in file_content:

            if len(chunk) < 2:
              continue

            date_of_dialog = chunk[0][0]
            qa_session = chunk[1]
            stenogram_link = chunk[2][0]

            if len(qa_session) < 2:
              continue

            # Будем считывать каждые 2 строки
            for index in range(0, len(qa_session), 2):
              question = qa_session[index].strip()
              answer = qa_session[index+1].strip()

              # вычислим косинусное расстояние между эмбеддингами
              cos_similarity = get_cosine_similarity(question, answer)

              # вычислим для вопроса и ответа их sentiment
              sentiment_dict_question = (dostoevsky_sentiment_model.predict([question]))[0]
              sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer]))[0]
              q_neutral = sentiment_dict_question['neutral']
              q_negative = sentiment_dict_question['negative']
              q_positive = sentiment_dict_question['positive']
              q_unsure = sentiment_dict_question['skip']
              q_colloquial = sentiment_dict_question['speech']
              a_neutral = sentiment_dict_answer['neutral']
              a_negative = sentiment_dict_answer['negative']
              a_positive = sentiment_dict_answer['positive']
              a_unsure = sentiment_dict_answer['skip']
              a_colloquial = sentiment_dict_answer['speech']


              # Добавим данные в датафрейм processed_data
              processed_data = processed_data.append({'Duma Convocation':session_name, \
              'Commitee name':commitee_name, 'Date': date_of_dialog,\
              'Question': question, \
              'Answer': answer,'Stenogram link':stenogram_link, 'Cosine Similarity' : cos_similarity,  \
              'q_neutral': q_neutral, 'q_negative': q_negative,  \
              'q_positive':q_positive, \
              'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
              'a_neutral':a_neutral, 'a_negative':a_negative, \
              'a_positive':a_positive, 'a_unsure':a_unsure, 
              'a_colloquial': a_colloquial},\
              ignore_index=True)
                
              # Добавим данные в датафрейм raw_data
              raw_data = raw_data.append({'Duma Convocation':session_name, \
                              'Commitee name':commitee_name,  'Date': date_of_dialog,\
                              'Question': question, 'Answer': answer, 'Stenogram link':stenogram_link}, ignore_index=True)
              



In [19]:
#processed_data.sort_values(by = 'Cosine Similarity', ascending = False)
processed_data[processed_data['Duma Convocation'] == 'duma 6'][:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Stenogram link,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,priro,20 февраля 2013 г,"Семён Романович, вот скажите, пожалуйста, поче...",Сегодня законопроект предполагает перевод земе...,http://transcript.duma.gov.ru/node/3805/,0.207537,0.793116,0.129413,0.03411,0.080367,0.025967,0.812877,0.156115,0.115971,0.037337,0.004478


In [ ]:
#pd.set_option('display.max_colwidth', None)

In [13]:
# Пример: считать ссылку с гитхаба

# url = 'https://github.com/AleksandraVasilieva/Scraping_stenograms/raw/main/dialogs%20data/session_6_all_dialogs_agrar.pkl'

# response = requests.get(url)
# import tempfile

# with tempfile.NamedTemporaryFile(delete=False) as temp_file:
#     temp_file.write(response.content)

#     # Access the filename using temp_file.name if needed

#     temp_file.seek(0)  # Reset file pointer to the beginning

#     loaded_list = pickle.load(temp_file)

# loaded_list[0]

In [20]:
raw_data[:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Stenogram link
0,duma 6,priro,20 февраля 2013 г,"Семён Романович, вот скажите, пожалуйста, поче...",Сегодня законопроект предполагает перевод земе...,http://transcript.duma.gov.ru/node/3805/


In [21]:
processed_data[:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Stenogram link,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,priro,20 февраля 2013 г,"Семён Романович, вот скажите, пожалуйста, поче...",Сегодня законопроект предполагает перевод земе...,http://transcript.duma.gov.ru/node/3805/,0.207537,0.793116,0.129413,0.03411,0.080367,0.025967,0.812877,0.156115,0.115971,0.037337,0.004478


In [22]:
# создадим датафрейм с добавочной информацией, где из текстов убираются имена

processed_data_nameless = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Date', 'Question', 'Answer', 'Stenogram link'\
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scraping_stenograms/dialogs data pickle format/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:

    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for path in files:

        commitee_name = path[len(path) - path[::-1].find('_'):][:-4]

        with open(path, 'rb') as file:
          file_content = pickle.load(file)

        # Пропустим файл, если он пустой
        if len(file_content) == 0:
          continue
          
        for chunk in file_content:

            if len(chunk) < 2:
              continue

            date_of_dialog = chunk[0][0]
            qa_session = chunk[1]
            stenogram_link = chunk[2][0]

            if len(qa_session) < 2:
              continue

            # Будем считывать каждые 2 строки
            for index in range(0, len(qa_session), 2):
              question = qa_session[index].strip()
              answer = qa_session[index+1].strip()

              question_nameless = nameless_text(question)
              answer_nameless = nameless_text(answer)

              # вычислим косинусное расстояние между эмбеддингами
              cos_similarity = get_cosine_similarity(question_nameless, answer_nameless)

              # вычислим для вопроса и ответа их sentiment
              sentiment_dict_question = (dostoevsky_sentiment_model.predict([question_nameless]))[0]
              sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer_nameless]))[0]
              q_neutral = sentiment_dict_question['neutral']
              q_negative = sentiment_dict_question['negative']
              q_positive = sentiment_dict_question['positive']
              q_unsure = sentiment_dict_question['skip']
              q_colloquial = sentiment_dict_question['speech']
              a_neutral = sentiment_dict_answer['neutral']
              a_negative = sentiment_dict_answer['negative']
              a_positive = sentiment_dict_answer['positive']
              a_unsure = sentiment_dict_answer['skip']
              a_colloquial = sentiment_dict_answer['speech']


              # Добавим данные в датафрейм processed_data_nameless
              processed_data_nameless = processed_data_nameless.append({'Duma Convocation':session_name, \
              'Commitee name':commitee_name, 'Date': date_of_dialog,\
              'Question': question, \
              'Answer': answer,'Stenogram link':stenogram_link,'Cosine Similarity' : cos_similarity,  \
              'q_neutral': q_neutral, 'q_negative': q_negative,  \
              'q_positive':q_positive, \
              'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
              'a_neutral':a_neutral, 'a_negative':a_negative, \
              'a_positive':a_positive, 'a_unsure':a_unsure, 
              'a_colloquial': a_colloquial},\
              ignore_index=True)


In [23]:
processed_data_nameless[:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Stenogram linkCosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial,Stenogram link,Cosine Similarity
0,duma 6,priro,20 февраля 2013 г,"Семён Романович, вот скажите, пожалуйста, поче...",Сегодня законопроект предполагает перевод земе...,NaN,0.782673,0.092698,0.037337,0.080367,0.025967,0.749097,0.168867,0.122533,0.050341,0.004342,http://transcript.duma.gov.ru/node/3805/,0.238637


In [24]:
processed_data_nameless[processed_data_nameless['Duma Convocation'] == 'duma 6'][:1]

,Duma Convocation,Commitee name,Date,Question,Answer,Stenogram linkCosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial,Stenogram link,Cosine Similarity
0,duma 6,priro,20 февраля 2013 г,"Семён Романович, вот скажите, пожалуйста, поче...",Сегодня законопроект предполагает перевод земе...,NaN,0.782673,0.092698,0.037337,0.080367,0.025967,0.749097,0.168867,0.122533,0.050341,0.004342,http://transcript.duma.gov.ru/node/3805/,0.238637


In [25]:
# raw_data.to_csv('raw_data.csv', index=False)
# processed_data.to_csv('processed_data.csv', index=False)
# processed_data_nameless.to_csv('processed_data_nameless.csv', index=False)